# Visualize weather data from https://www.earthsystemgrid.org/dataset/isabeldata.html

Written by Jean M Favre, Swiss National Supercomputing Center.
Tested on Piz Daint Tue May 11 18:00:49 CEST 2021

In [ ]:
import gzip
import numpy as np

# Details about file format are here:

https://www.earthsystemgrid.org/dataset/isabeldata/file/readme.html

http://vis.computer.org/vis2004contest/data.html

Data Format

A summary of the data can be found below:

    Dimensions: 500 x 500 x 100
    Physical Scale: 2139km (east-west) x 2004km (north-south) x 19.8km (vertical) Note: The east-west distance of 2139km is measured at the southern latitude; the distance is only 1741km at the northern latitude due to the curvature of the earth.
    Physical Location: Longitude (x): 83W to 62W; Latitude (y): 23.7N to 41.7N; Height (z): 0.035km to 19.835km
    Format: Brick-of-Floats

In [ ]:
dims = [500,500,100] # original dimensions of binary data in disk file

def load_volume(varname, frame_number):
  assert frame_number >= 1 and frame_number <= 48
  fname_gzipped = format('/scratch/snx3000/jfavre/Isabel/%s%02d.bin.gz' % (varname, frame_number))
  print("opening ", fname_gzipped)
  with gzip.open(fname_gzipped,'rb') as f:
    data = np.frombuffer(f.read(), dtype='>f4')
    # Land values, where there is no valid atmospheric data, are marked 1.0e35
    if varname == 'PRECIPf':
      data = np.where(data < 3e35, data, np.NaN)
    elif varname == 'CLOUDf':
      data = np.where(data < 2e35, data, np.NaN)
    else:
      data = np.where(data != 1e35, data, np.NaN)
  return data.reshape(np.flip(dims))

# Reading it with a ParaView programmable Source

In [ ]:
from paraview.simple import *

In [ ]:
view = GetRenderView()
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(view)
w = display(pvdisp)

In [ ]:
RequestData = """
import numpy as np
executive = self.GetExecutive()
outInfo = executive.GetOutputInformation(0)
exts = [executive.UPDATE_EXTENT().Get(outInfo, i) for i in range(6)]
dims = (exts[1]-exts[0]+1, exts[3]-exts[2]+1, exts[5]-exts[4]+1)
output.SetExtent(exts)

timestep = 24

for v in ['CLOUDf', 'Pf', 'PRECIPf', 'QCLOUDf', 'QGRAUPf','QICEf','QRAINf','QSNOWf','QVAPORf','TCf','Uf','Vf','Wf']:
  output.PointData.append(load_volume(v, timestep).ravel(), v)
"""

RequestInfo = """
dims = [500,500,100]
executive = self.GetExecutive ()
outInfo = executive.GetOutputInformation(0)

outInfo.Set(executive.WHOLE_EXTENT(), 0, dims[0]-1 , 0, dims[1]-1 , 0, dims[2]-1)
outInfo.Set(vtk.vtkDataObject.SPACING(), 1, 1, 1)
outInfo.Set(vtk.vtkDataObject.ORIGIN(), 0,0,0)
"""

# create a new 'Programmable Source'
IsabelGrid = ProgrammableSource()
IsabelGrid.OutputDataSetType = 'vtkImageData'
IsabelGrid.Script = RequestData
IsabelGrid.ScriptRequestInformation = RequestInfo
IsabelGrid.UpdatePipeline()

rep1 = Show(IsabelGrid, GetRenderView())

In [ ]:
rep1.Representation = "Surface"
ColorBy(rep1, ('POINTS', 'QVAPORf'))
ResetCamera()

In [ ]:
rep1.Representation = "Volume"

In [ ]:
rep1.Shade=1

# Let us now see the Python Calculator
Here, inputs[0] refer to the first input (dataset) to the filter. The Python Calculator can accept multiple inputs. Each input can be accessed as inputs[0] , inputs[1] , … You can access the point or cell data of an input using the .PointData or .CellData qualifiers. You can then access individual arrays within the point or cell data containers using the [] operator. Make sure to use quotes or double-quotes around the array name

In [ ]:
# Accessing data
pyCalculator = PythonCalculator(Input=IsabelGrid)
pyCalculator.Expression = "inputs[0].PointData['QVAPORf']"
pyCalculator.ArrayName = "result"
pyCalculator.ArrayAssociation = 'Point Data'

In [ ]:
pyCalculator.Expression = "inputs[0].PointData['QVAPORf'] * 10"
pyCalculator.UpdatePipeline()

In [ ]:
# we have created a new scalar field called "result"
# equal to 10 times the scalar array 'QVAPORf'
pyCalculator.PointData[:]

In [ ]:
pyCalculator.PointData['QVAPORf'].GetRange()

In [ ]:
pyCalculator.PointData['result'].GetRange()

In [ ]:
# do a division, element by element
pyCalculator2 = PythonCalculator(Input = pyCalculator)
pyCalculator2.Expression = "inputs[0].PointData['QVAPORf'] / inputs[0].PointData['result']"
pyCalculator2.ArrayName = "result"
pyCalculator2.UpdatePipeline()

In [ ]:
pyCalculator2.PointData['result'].GetRange()

In [ ]:
# normalize the field 'QVAPOR'
pyCalculator3 = PythonCalculator(Input = IsabelGrid)
pyCalculator3.Expression = "inputs[0].PointData['QVAPORf'] / np.nanmax(inputs[0].PointData['QVAPORf'])"
pyCalculator3.ArrayName = "result"
pyCalculator3.UpdatePipeline()

In [ ]:
pyCalculator3.PointData['result'].GetRange()

In [ ]:
# make a vector (3-tuple) with U, V and W component of Wind velocity
pyCalculator4 = PythonCalculator(Input = IsabelGrid)
pyCalculator4.Expression = "make_vector(inputs[0].PointData['Uf'],inputs[0].PointData['Vf'], inputs[0].PointData['Wf'])"
pyCalculator4.ArrayName = "Wind"
pyCalculator4.UpdatePipeline()

In [ ]:
# Calculate the magnitude of Wind velocity
pyCalculator5 = PythonCalculator(Input = IsabelGrid)
pyCalculator5.Expression = "mag(make_vector(inputs[0].PointData['Uf'],inputs[0].PointData['Vf'], inputs[0].PointData['Wf']))"
pyCalculator5.ArrayName = "Wind Magnitude"
pyCalculator5.UpdatePipeline()

In [ ]:
# Range of magnitude of Wind
pyCalculator5.PointData['Wind Magnitude'].GetRange()